In [ ]:
#import pandas as pd
#data = pd.read_csv('ภาคกลาง.csv', encoding='ANSI',low_memory=False)
#data = data[data['ref_province_id']==18]
#data.to_csv('Prov18_ชัยนาท_จปฐ_1.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd
import numpy as np

cols_as_str = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
data = pd.read_csv(r"C:\Users\NSO\Desktop\pop_run_data\add_data\r_alldata_test_202509051021.csv",dtype={col: str for col in cols_as_str}, encoding='utf-8-sig',keep_default_na=False,low_memory=False)
data_cdd = pd.read_csv(r"C:\Users\NSO\Desktop\pop_run_data\add_data\Prov18_cpp.csv",dtype={col: str for col in cols_as_str}, encoding='utf-8-sig',keep_default_na=False,low_memory=False)

cols_to_zfill = {
    'RegCode': 1,
    'ProvCode': 2,
    'DistCode': 2,
    'SubDistCode': 2,
    'VilCode': 2
}

# จัดการทุกคอลัมน์ตามที่ระบุ
for col, width in cols_to_zfill.items():
    if col in data.columns:
        data[col] = data[col].apply(
            lambda x: '' if pd.isna(x) or str(x).strip() == '' else str(int(float(str(x).strip()))).zfill(width)
        )

for col, width in cols_to_zfill.items():
    if col in data.columns:
        data_cdd[col] = data_cdd[col].apply(
            lambda x: '' if pd.isna(x) or str(x).strip() == '' else str(int(float(str(x).strip()))).zfill(width)
        )

# จัดการ HouseNumber ให้เป็น string อย่างเดียว
if 'HouseNumber' in data.columns:
    data['HouseNumber'] = data['HouseNumber'].fillna('').astype(str).str.strip()

if 'HouseNumber' in data.columns:
    data_cdd['HouseNumber'] = data_cdd['HouseNumber'].fillna('').astype(str).str.strip()


data['Household_No_new'] = data['Household_No']
mask_blank = data['Household_No'].isna() | (data['Household_No'].astype(str).str.strip() == '')
data.loc[mask_blank, 'Household_No_new'] = list(range(1, mask_blank.sum() + 1))

data = data.sort_values(by=['NsoBuilding_No','Building_No','RegCode', 'ProvCode', 'DistCode', 'SubDistCode', 'AreaCode','MunTaoCode','VilCode',
                            'EA_Code_15', 'Household_No','Household_No_new', 'HouseNumber','RoomNumber','HouseholdMemberNumber'], ascending=True).reset_index(drop=True)

data_cdd = data_cdd.sort_values(by=['RegCode','ProvCode','DistCode', 'SubDistCode', 'VilCode', 'HouseNumber', 'Household_No','Population_No'], ascending=True).reset_index(drop=True)
############################################################################################################################################
df = data.copy().reset_index(drop=True)
df_cdd = data_cdd.copy().reset_index(drop=True)

key_columns_df = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
df['key_HH_df'] = df[key_columns_df].astype(str).agg('|'.join, axis=1)

key_columns_df_cdd = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
df_cdd['key_HH_df_cdd'] = df_cdd[key_columns_df_cdd].astype(str).agg('|'.join, axis=1)

df_HH = df.groupby('Household_No_new')[['key_HH_df']].first().reset_index()
df_cdd_HH = df_cdd.groupby('Household_No')[['key_HH_df_cdd']].first().reset_index()

df_cdd_Add = df_cdd_HH[~df_cdd_HH['key_HH_df_cdd'].isin(df_HH['key_HH_df'])]


merged_df = pd.merge(df_cdd, df_cdd_Add[['Household_No']], on='Household_No', how='left', indicator=True)
df_cdd['Merge_Flag'] = (merged_df['_merge'] == 'both').astype(int).reset_index(drop=True)
df_cdd = df_cdd[df_cdd['Merge_Flag'] == 1]
df_cdd_final = df_cdd.copy().reset_index(drop=True)

df_cdd_final.drop(columns=['key_HH_df_cdd','Merge_Flag'], inplace=True)
df.drop(columns=['Household_No_new','key_HH_df'], inplace=True)


df_cdd_final['DataSources'] = 4


df_final_2 = pd.concat([df, df_cdd_final], axis=0, ignore_index=True)
df_final_2['HouseNumber'] = df_final_2['HouseNumber'].apply(lambda x: f'="{x}"')
df_final_2.to_csv('df_final_ชัยนาท_2.csv', index=False, encoding='utf-8-sig')



# import pandas as pd
# import numpy as np

# cols_as_str = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
# data = pd.read_csv(r"C:\Users\NSO\Desktop\pop_run_data\add_data\df_final_18_test.csv",dtype={col: str for col in cols_as_str}, encoding='utf-8-sig',keep_default_na=False,low_memory=False)
# data_cdd = pd.read_csv(r"C:\Users\NSO\Desktop\pop_run_data\add_data\Prov18_cpp_test.csv",dtype={col: str for col in cols_as_str}, encoding='utf-8-sig',keep_default_na=False,low_memory=False)

# cols_to_zfill = {
#     'RegCode': 1,
#     'ProvCode': 2,
#     'DistCode': 2,
#     'SubDistCode': 2,
#     'VilCode': 2
# }

# # จัดการทุกคอลัมน์ตามที่ระบุ
# for col, width in cols_to_zfill.items():
#     if col in data.columns:
#         data[col] = data[col].apply(
#             lambda x: '' if pd.isna(x) or str(x).strip() == '' else str(int(float(str(x).strip()))).zfill(width)
#         )

# for col, width in cols_to_zfill.items():
#     if col in data.columns:
#         data_cdd[col] = data_cdd[col].apply(
#             lambda x: '' if pd.isna(x) or str(x).strip() == '' else str(int(float(str(x).strip()))).zfill(width)
#         )

# # จัดการ HouseNumber ให้เป็น string อย่างเดียว
# if 'HouseNumber' in data.columns:
#     data['HouseNumber'] = data['HouseNumber'].fillna('').astype(str).str.strip()

# if 'HouseNumber' in data.columns:
#     data_cdd['HouseNumber'] = data_cdd['HouseNumber'].fillna('').astype(str).str.strip()


# data['Household_No_new'] = data['Household_No']
# mask_blank = data['Household_No'].isna() | (data['Household_No'].astype(str).str.strip() == '')
# data.loc[mask_blank, 'Household_No_new'] = list(range(1, mask_blank.sum() + 1))

# data = data.sort_values(by=['NsoBuilding_No','Building_No','RegCode', 'ProvCode', 'DistCode', 'SubDistCode', 'AreaCode','MunTaoCode','VilCode',
#                             'EA_Code_15', 'Household_No','Household_No_new', 'HouseNumber','RoomNumber','HouseholdMemberNumber'], ascending=True).reset_index(drop=True)

# data_cdd = data_cdd.sort_values(by=['RegCode','ProvCode','DistCode', 'SubDistCode', 'VilCode', 'HouseNumber', 'Household_No','Population_No'], ascending=True).reset_index(drop=True)
# ############################################################################################################################################
# df = data.copy().reset_index(drop=True)
# df_cdd = data_cdd.copy().reset_index(drop=True)

# key_columns_df = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
# df['key_HH_df'] = df[key_columns_df].astype(str).agg('|'.join, axis=1)

# key_columns_df_cdd = ['RegCode','ProvCode','DistCode','SubDistCode','VilCode','HouseNumber']
# df_cdd['key_HH_df_cdd'] = df_cdd[key_columns_df_cdd].astype(str).agg('|'.join, axis=1)

# df_HH = df.groupby('Household_No_new')[['key_HH_df']].first().reset_index()
# df_cdd_HH = df_cdd.groupby('Household_No')[['key_HH_df_cdd']].first().reset_index()

# df_cdd_Add = df_cdd_HH[~df_cdd_HH['key_HH_df_cdd'].isin(df_HH['key_HH_df'])]


# merged_df = pd.merge(df_cdd, df_cdd_Add[['Household_No']], on='Household_No', how='left', indicator=True)
# df_cdd['Merge_Flag'] = (merged_df['_merge'] == 'both').astype(int).reset_index(drop=True)
# df_cdd = df_cdd[df_cdd['Merge_Flag'] == 1]
# df_cdd_final = df_cdd.copy().reset_index(drop=True)

# df_cdd_final.drop(columns=['key_HH_df_cdd','Merge_Flag'], inplace=True)
# df.drop(columns=['Household_No_new','key_HH_df'], inplace=True)


# df_cdd_final['DataSources'] = 4


# df_final_2 = pd.concat([df, df_cdd_final], axis=0, ignore_index=True)

# # --- ส่วนที่แก้ไขและเพิ่มเข้ามาสำหรับทำสีและส่งออกเป็น Excel ---

# # 1. กำหนดชื่อไฟล์ผลลัพธ์เป็น .xlsx
# output_filename = 'df_final_ชัยนาท_styled.xlsx'

# # 2. สร้างฟังก์ชันเพื่อกำหนดสไตล์ (สีพื้นหลัง)
# def highlight_new_rows(row):
#     # ตรวจสอบว่า index ของแถวปัจจุบัน มากกว่าหรือเท่ากับจำนวนแถวของข้อมูลตั้งต้น (df) หรือไม่
#     # ถ้าใช่ แสดงว่าเป็นแถวที่เพิ่มเข้ามาใหม่
#     if row.name >= len(df):
#         # คืนค่าสไตล์ 'background-color: orange' สำหรับทุกคอลัมน์ในแถวนั้น
#         return ['background-color: orange'] * len(row)
#     else:
#         # ถ้าเป็นแถวเดิม ไม่ต้องใส่สไตล์
#         return [''] * len(row)

# # 3. ใช้ Styler เพื่อนำฟังก์ชันไปใช้กับ DataFrame และสร้างไฟล์ Excel
# print(f"กำลังสร้างไฟล์ Excel '{output_filename}' พร้อมใส่สี...")

# # สร้าง Styler object และใช้ฟังก์ชัน highlight_new_rows กับทุกแถว (axis=1)
# styled_df = df_final_2.style.apply(highlight_new_rows, axis=1)

# # ส่งออก Styler object เป็นไฟล์ Excel
# styled_df.to_excel(
#     output_filename,
#     engine='openpyxl',
#     index=False,
# )

# print(f"สร้างไฟล์ '{output_filename}' เรียบร้อยแล้ว")
# # --- สิ้นสุดส่วนที่แก้ไข ---